Codul de mai jos este codul prin care am reusit dintr-o imagine auxiliara sa imi scot toate 
templaturile necesare pentru determinarea corecta a cifrelor

Acest cod nu necesita rulare, deoarce directorul **templates** are deja imaginea auxiliara prelucrata in 
templates ci este doar o dovada a acestui lucru.

In [ ]:
import cv2 as cv
import numpy as np
import os
from copy import deepcopy

In [ ]:
def extrage_careu(image):
    #transformam imaginea in imagine HSV
    img_hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    #vrem sa luam din imagine tabla albastra
    #valorile le-am gasit jucandu-ma cu functia din lab find_color_values_using_trackbar
    low_blue=(81, 0, 0)
    high_blue=(119, 255, 255)
    mask_blue_hsv = cv.inRange(img_hsv, low_blue,high_blue)
    #scapam de o parte din zgomot
    kernel = np.ones((3, 3), np.uint8)
    thresh = cv.erode(mask_blue_hsv, kernel)
    #show_image('image_thresholded',thresh)

    edges =  cv.Canny(thresh ,50,150)
    # show_image('edges',edges, 0.15,0.15)
    contours, _ = cv.findContours(edges,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    max_area = 0

    for i in range(len(contours)):
        if(len(contours[i]) >3):
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis = 1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left

    width = 2000
    height = 2000

    image_copy = image.copy()
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)
    # show_image("detected corners",image_copy,0.15,0.15)

    puzzle = np.array([top_left,top_right,bottom_right,bottom_left], dtype = "float32")
    destination_of_puzzle = np.array([[0,0],[width,0],[width,height],[0,height]], dtype = "float32")
    M = cv.getPerspectiveTransform(puzzle,destination_of_puzzle)
    result = cv.warpPerspective(image, M, (width, height))
    cv.imwrite('./decupaj.jpg',result)
    #scoatem tabla cea mica 
    width = 80 * 14
    height = 80 * 14
    top_right = (1738, 266)
    top_left = (264, 264)
    bottom_right = (1739, 1748)
    bottom_left = (257, 1741)
    # top_right = (1695, 260)
    # top_left = (260, 255)
    # bottom_right = (1695, 1705)
    # bottom_left = (250, 1695)

    puzzle = np.array([top_left, top_right, bottom_left, bottom_right], dtype="float32")
    destination_of_puzzle = np.array([[0,0], [width, 0], [0, height], [width, height]], dtype="float32")
    M = cv.getPerspectiveTransform(puzzle, destination_of_puzzle)
    result = cv.warpPerspective(result, M, (width, height))

    return result

In [ ]:
def get_patch(image, row, col, off1, off2, off3, off4):
    patch = image[row * 80 + off1 : (row + 1) * 80 - off2, col * 80 + off3: (col + 1) * 80 - off4]
    return patch

In [ ]:
img_name = f'./imagini_auxiliare/04.jpg'
img = cv.imread(img_name)
img_cropped= extrage_careu(img)
# show_image("gh",img_cropped)

In [ ]:
def scoate_templaturi(nr,row,col,top,bottom,left,right):
    kernel = np.ones((2, 2), np.uint8)
    patch = get_patch(img_cropped, row, col, top, bottom, left, right)
    _, thresh = cv.threshold(cv.cvtColor(patch, cv.COLOR_BGR2GRAY), 100, 255, cv.THRESH_BINARY_INV)
    dilated = cv.erode(thresh, kernel)
    dilated = cv.dilate(dilated, kernel)

    tmp_name = f'./templates/{nr}.jpg'
    cv.imwrite(tmp_name, dilated)

In [ ]:
 # scoate_templaturi(0, 0, 0, 10, 17, 15, 11)
# scoate_templaturi(1,0,2,25,5,10,10)
# scoate_templaturi(2,0,4,22,12,6,8)
# scoate_templaturi(3,0,6,21,13,10,10)
# scoate_templaturi(4,0,8,23,15,7,7)
# scoate_templaturi(5,0,10,22,12,7,7)
# scoate_templaturi(6,0,12,22,10,12,12)
# scoate_templaturi(7,2,0,22,12,7,7)
# scoate_templaturi(8,2,2,24,10,12,12)
# scoate_templaturi(9, 2, 4, 21, 13, 7, 7)
# scoate_templaturi(10, 2, 6, 22, 13, 17, 10)
# scoate_templaturi(11, 2, 8, 22, 13, 15, 18)
# scoate_templaturi(12, 2, 10, 19, 14, 12, 14)
# scoate_templaturi(13, 2, 12, 19, 14, 16, 12)
# scoate_templaturi(14, 4, 0, 20, 15, 15, 12)
# scoate_templaturi(15, 4, 2, 19, 14, 14, 10)
# scoate_templaturi(16, 4, 4, 18, 14, 20, 8)
# scoate_templaturi(17, 4, 6, 19, 14, 20, 9)
# scoate_templaturi(18, 4, 8, 20, 14, 13, 11)
# scoate_templaturi(19, 4, 10, 20, 14, 13, 11)
# scoate_templaturi(20, 4, 12, 20, 12, 7, 11)
# scoate_templaturi(21, 6, 0, 20, 13, 15, 11)
# scoate_templaturi(24, 6, 2, 19, 14, 15, 4)
# scoate_templaturi(25, 6, 4, 19, 14, 14, 8)
# scoate_templaturi(27, 6, 6, 11, 21, 16, 7)
# scoate_templaturi(28, 6, 8, 11, 21, 6, 14)
# scoate_templaturi(30, 6, 10, 11, 26, 10, 11)
# scoate_templaturi(32, 6, 12, 13, 20, 9, 8)
# scoate_templaturi(35, 8, 0, 25, 8, 10, 10)
# scoate_templaturi(36, 8, 2, 19, 14, 12, 10)
# scoate_templaturi(40, 8, 4, 20, 14, 10, 5)
# scoate_templaturi(42, 8, 6, 19, 15, 12, 6)
# scoate_templaturi(45, 8, 8, 19, 15, 10, 8)
# scoate_templaturi(48, 8, 10, 17, 18, 13, 6)
# scoate_templaturi(49, 8, 12, 19, 16, 11, 6)
# scoate_templaturi(50, 10, 0, 21, 11, 9, 10)
# scoate_templaturi(54, 10, 2, 17, 13, 7, 8)
# scoate_templaturi(56, 10, 4, 19, 13, 14, 6)
# scoate_templaturi(60, 10, 6, 16, 18, 15, 5)
# scoate_templaturi(63, 10, 8, 19, 14, 11, 7)
# scoate_templaturi(64, 10, 10, 20, 10, 7, 5)
# scoate_templaturi(70, 10, 12, 20, 14, 12, 7)
# scoate_templaturi(72, 12, 0, 27, 7, 13, 7)
# scoate_templaturi(80, 12, 2, 22, 9, 12, 5)
# scoate_templaturi(81, 12, 4, 20, 12, 19, 13)
# scoate_templaturi(90, 12, 6, 18, 15, 13, 5)